In [49]:
import torch
a = torch.tensor([[[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                  [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
a.transpose_(0, 1)
a = a.contiguous()
a = a.view(2, -1)
a.shape

torch.Size([2, 9])

In [47]:
?a.view

Docstring:
view(*shape) -> Tensor

Returns a new tensor with the same data as the :attr:`self` tensor but of a
different :attr:`shape`.

The returned tensor shares the same data and must have the same number
of elements, but may have a different size. For a tensor to be viewed, the new
view size must be compatible with its original size and stride, i.e., each new
view dimension must either be a subspace of an original dimension, or only span
across original dimensions :math:`d, d+1, \dots, d+k` that satisfy the following
contiguity-like condition that :math:`\forall i = d, \dots, d+k-1`,

.. math::

  \text{stride}[i] = \text{stride}[i+1] \times \text{size}[i+1]

Otherwise, it will not be possible to view :attr:`self` tensor as :attr:`shape`
without copying it (e.g., via :meth:`contiguous`). When it is unclear whether a
:meth:`view` can be performed, it is advisable to use :meth:`reshape`, which
returns a view if the shapes are compatible, and copies (equivalent to calling
:meth:`conti

In [7]:
torch.eye(3)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [18]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("kind", type=str,
                    help="Kind of experiment: single-model, hidden-loop or hidden-sample")
?parser

Type:        ArgumentParser
String form: ArgumentParser(prog='ipykernel_launcher.py', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)
File:        ~/miniconda3/lib/python3.10/argparse.py
Docstring:  
Object for parsing command line strings into Python objects.

Keyword Arguments:
    - prog -- The name of the program (default:
        ``os.path.basename(sys.argv[0])``)
    - usage -- A usage message (default: auto-generated from arguments)
    - description -- A description of what the program does
    - epilog -- Text following the argument descriptions
    - parents -- Parsers whose arguments should be copied into this one
    - formatter_class -- HelpFormatter class for printing help messages
    - prefix_chars -- Characters that prefix optional arguments
    - fromfile_prefix_chars -- Characters that prefix files containing
        additional arguments
    - argument_default -- The default value for all argume

In [ ]:
from sklearn.datasets import load_svmlight_file, make_blobs

Object `sklearn.datasets` not found.


In [3]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self, input_dim=3, output_dim=1):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim, dtype=X.dtype, bias=False)
    def forward(self, x):
        y = self.linear(x)
        return y
    
model = LinearRegressionModel(input_dim=X.shape[1], output_dim=y.shape[1])
print(model)
criterion = torch.nn.MSELoss()

LinearRegressionModel(
  (linear): Linear(in_features=3, out_features=2, bias=False)
)


In [12]:
Q, L, _ = torch.svd(model.linear.weight.T @ model.linear.weight)
Q @ torch.diag(L) @ Q.T

tensor([[0.3128, 0.3760, 0.1549],
        [0.3760, 0.5122, 0.2396],
        [0.1549, 0.2396, 0.1242]], grad_fn=<MmBackward0>)

In [9]:
model.linear.weight.T @ model.linear.weight

tensor([[0.3128, 0.3760, 0.1549],
        [0.3760, 0.5122, 0.2396],
        [0.1549, 0.2396, 0.1242]], grad_fn=<MmBackward0>)

In [ ]:
from importlib import reload
import soap
import new_soap
reload(soap)
reload(new_soap)

model = LinearRegressionModel(input_dim=X.shape[1], output_dim=y.shape[1])
# optimizer = soap.SOAP(model.parameters(), 
#                  lr=3e-3, 
#                  betas=(.95, .95), 
#                  weight_decay=.01, 
#                  precondition_frequency=10)
optimizer = new_soap.NEW_SOAP(model.parameters(), 
                 lr=3e-3, 
                 beta=.95, 
                 weight_decay=.01, 
                 precondition_frequency=10)
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [83]:
num_epochs = 3
num_verbose = 1
for epoch in range(num_epochs):
    pred_y = model(X)
 
    loss = criterion(pred_y, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % num_verbose == 0:
        print('Epoch {}: loss {}'.format(epoch, loss.item()))

KeyError: 'beta'